In [1]:
import os
import math
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

from utils import *

In [8]:
d = {
    'id': [1, 2, 3, 4], 
    'longitude': [-73.589462, -73.590055, -73.590041, -73.589008],
    'latitude': [45.592012, 45.592189, 45.592181, 45.590781],
    'poly_id': [99, 99, 99, 99]
}
df = pd.DataFrame(data=d)

gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf

,id,longitude,latitude,poly_id,geometry
0,1,-73.589462,45.592012,99,POINT (-73.58946 45.59201)
1,2,-73.590055,45.592189,99,POINT (-73.59006 45.59219)
2,3,-73.590041,45.592181,99,POINT (-73.59004 45.59218)
3,4,-73.589008,45.590781,99,POINT (-73.58901 45.59078)


In [3]:
from scipy.spatial import cKDTree
import numpy as np

# Create a KDTree from the geometry of the GeoDataFrame
tree = cKDTree(np.array(gdf.geometry.apply(lambda geom: [geom.x, geom.y])).tolist())

# Query the tree for the closest points to each point in the GeoDataFrame
distances, indices = tree.query(np.array(gdf.geometry.apply(lambda geom: [geom.x, geom.y])).tolist(), k=2)

# Get the closest point for each row
gdf['closest_point'] = gdf.geometry.iloc[indices[:, 1]].values


In [4]:
gdf

,id,longitude,latitude,geometry,closest_point
0,1,-73.589462,45.592012,POINT (-73.58946 45.59201),POINT (-73.59004 45.59218)
1,2,-73.590055,45.592189,POINT (-73.59006 45.59219),POINT (-73.59004 45.59218)
2,3,-73.590041,45.592181,POINT (-73.59004 45.59218),POINT (-73.59006 45.59219)
3,4,-73.589008,45.590781,POINT (-73.58901 45.59078),POINT (-73.58946 45.59201)


In [5]:
m = folium.Map(location=[45.5414195, -73.6303031], tiles="cartodbpositron", zoom_start=12)

In [6]:
for _, row in gdf.iterrows():
    lat = row.latitude
    lon = row.longitude
    folium.Marker(location=[lat, lon], popup=row["id"]).add_to(m)

In [7]:
m